In [27]:
!pip install factor_analyzer

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/work_space/Project/ABFR')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('./Data/house-prices-advanced-regression-techniques/train.csv')
numeric_df = df.select_dtypes(include=['int', 'float'])

In [3]:
numeric_df = numeric_df.dropna(axis=0)
numeric_df.reset_index(inplace=True, drop=True)

In [ ]:
numeric_df.iloc[:, 1:30].corr()

In [5]:
array_df = np.array(numeric_df.iloc[:, 1:30])
array_df = np.round(array_df, 3)
array_df

array([[   60.,    65.,  8450., ...,   548.,     0.,    61.],
       [   20.,    80.,  9600., ...,   460.,   298.,     0.],
       [   60.,    68., 11250., ...,   608.,     0.,    42.],
       ...,
       [   70.,    66.,  9042., ...,   252.,     0.,    60.],
       [   20.,    68.,  9717., ...,   240.,   366.,     0.],
       [   20.,    75.,  9937., ...,   276.,   736.,    68.]])

In [6]:
from util.module import *

In [7]:
fa = Factor_attention(array_df, 25, 3)
fa.col_to_vec()

In [11]:
fa.under_cos_sim

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.80805919, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.55285903, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.80805919, 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.51803176, 0.        , 0.        ,
  

[[0],
 [1, 2, 12],
 [1, 2, 12, 23],
 [3, 5, 6, 7, 11, 12, 15, 18, 22, 23, 24, 25, 26, 27, 28],
 [4],
 [3, 5, 6, 7, 18, 24, 25],
 [3, 5, 6, 18, 24, 25],
 [3, 5, 7, 11, 12, 15, 22, 23, 25, 26],
 [8, 11, 16],
 [9],
 [10],
 [3, 7, 8, 11, 12, 26, 28],
 [1, 2, 3, 7, 11, 12, 15, 23, 26],
 [13, 15, 19, 20, 22],
 [14],
 [3, 7, 12, 13, 15, 18, 20, 22, 23, 25, 26, 28],
 [8, 16],
 [17],
 [3, 5, 6, 15, 18, 22, 24, 25, 28],
 [13, 19],
 [13, 15, 20, 22],
 [21],
 [3, 7, 13, 15, 18, 20, 22, 23],
 [2, 3, 7, 12, 15, 22, 23],
 [3, 5, 6, 18, 24, 25, 26],
 [3, 5, 6, 7, 15, 18, 24, 25, 26],
 [3, 7, 11, 12, 15, 24, 25, 26, 28],
 [3, 27],
 [3, 11, 15, 18, 26, 28]]

In [ ]:
import numpy as np

# 결과를 저장할 리스트 초기화

def sim_result(array):
  result_lists = []

  # 각 행별로 값이 있는 위치의 인덱스를 찾아 추출
  for row in array:
      indices = np.where(row != 0)[0]  # 0이 아닌 값의 인덱스 추출
      result_lists.append(indices.tolist())

  return result_lists

In [ ]:
sim_result(temp_array)

[[0], [1, 2, 3], [1, 2, 3], [1, 2, 3], [4]]

In [ ]:
temp = TSNE_attention(array_df, 3, 5)

In [ ]:
temp.tsne_data.shape

(1121, 3)

In [ ]:
temp2 = PCA_attention(array_df, 5, 5)
temp2.pca_data.shape

(1121, 5)